In [1]:
import string, re, json, pickle, warnings, nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
# importing the wordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from bidi.algorithm import get_display
import arabic_reshaper
from data_preparation import clean_content_aravec, clean

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
#df= pd.read_csv("offenssivelabelfinal.csv")

In [3]:
df1= pd.read_csv("train.csv")
df2= pd.read_csv("test.csv")

In [55]:
#df1, df2= train_test_split(df, test_size =.2, random_state=100)

In [56]:
df1

,Unnamed: 0,comment,off
0,2908,فنانين الكبت والفساد .عقلية جنسية لا غير. العف...,1
1,1055,الدعارة هربت منها في المحمدية و سكنت في بوزنيق...,1
2,181,كون غير خريتي و مدرتيش هادشي,1
3,4313,لا حول ولا قوة الا بالله العلي العظيم لا حول و...,0
4,228,الله يرحم الوالدين عندي مشكل كبيييير و محتاجة ...,0
...,...,...,...
6414,79,هدى جمعاك شعر كحل,0
6415,3927,تفو.. على أغنية زملاوية بحال مولاها,1
6416,5955,كون خرجات ها.ي كالت انا كنشطح كون بان ليك مساع...,1
6417,6936,الفديو زوين.,0


In [57]:
df2

,Unnamed: 0,comment,off
0,6139,تبهديل تسؤلو البنات في الشارع,1
1,3447,انا مستعد بي مسعدة بسيطة في وجه اللة ربي يادير...,0
2,6844,لاحول ولاقوة الا بالله العلي العظيم,0
3,2512,حتى هاد العائلة لي جالس وسط منهوم خص يدلمهوم ا...,1
4,2000,لا حول ولا قوة الا بالله اش هذا المستوى اصلا ر...,1
...,...,...,...
1600,6470,لي جا من عند بيمو اخبط لايك 😹😹😹😹,0
1601,6198,ونتساءل لمادا الله ارسل لنا فيروس كورونا,1
1602,3009,خانزة كيلوط وا الرويجل سير دير الرقية نصحك هاد...,1
1603,647,سير الله يعفو اعليك,0


In [59]:
#df1.head()

In [60]:
comments = ' '.join(list(df.comment))
comments

NameError: name 'df' is not defined

In [8]:
df.columns

Index(['comment', 'off'], dtype='object')

In [61]:
df1["clean_comment"]=df1["comment"].apply(lambda row: clean(row))
df2["clean_comment"]=df2["comment"].apply(lambda row: clean(row))

In [62]:
df1["clean_comment"][8]

'نوع اخر البشر زوامل الشوهه'

In [63]:
def replace_emoji(sent):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

In [66]:
replace_emoji(df1["clean_comment"][8])

'نوع اخر البشر زوامل الشوهه'

In [67]:
df1["clean_comment_nemoji"]=df1["clean_comment"].apply(lambda row: replace_emoji(row))
df2["clean_comment_nemoji"]=df2["clean_comment"].apply(lambda row: replace_emoji(row))

In [68]:
df1["clean_comment_nemoji"][8]

'نوع اخر البشر زوامل الشوهه'

In [69]:
df1

,Unnamed: 0,comment,off,clean_comment,clean_comment_nemoji
0,2908,فنانين الكبت والفساد .عقلية جنسية لا غير. العف...,1,فنانين الكبت والفساد عقليه جنسيه العفن الفن شع...,فنانين الكبت والفساد عقليه جنسيه العفن الفن شع...
1,1055,الدعارة هربت منها في المحمدية و سكنت في بوزنيق...,1,الدعاره هربت المحمديه سكنت بوزنيقه بحال بحال م...,الدعاره هربت المحمديه سكنت بوزنيقه بحال بحال م...
2,181,كون غير خريتي و مدرتيش هادشي,1,كون خريتي مدرتيش هادشي,كون خريتي مدرتيش هادشي
3,4313,لا حول ولا قوة الا بالله العلي العظيم لا حول و...,0,بالله العلي العظيم بالله العلي العظيم استغفرال...,بالله العلي العظيم بالله العلي العظيم استغفرال...
4,228,الله يرحم الوالدين عندي مشكل كبيييير و محتاجة ...,0,الله يرحم الوالدين عندي مشكل كبيير محتاجه الدع...,الله يرحم الوالدين عندي مشكل كبيير محتاجه الدع...
...,...,...,...,...,...
6414,79,هدى جمعاك شعر كحل,0,هدي جمعاك شعر كحل,هدي جمعاك شعر كحل
6415,3927,تفو.. على أغنية زملاوية بحال مولاها,1,تفو اغنيه زملاويه بحال مولاها,تفو اغنيه زملاويه بحال مولاها
6416,5955,كون خرجات ها.ي كالت انا كنشطح كون بان ليك مساع...,1,كون خرجات هاي كالت انا كنشطح كون ليك مساعادات ...,كون خرجات هاي كالت انا كنشطح كون ليك مساعادات ...
6417,6936,الفديو زوين.,0,الفديو زوين,الفديو زوين


<h2>splitting the data into target and feature</h2>

In [70]:
X_train = df1.clean_comment_nemoji
Y_train = df1.off
X_test = df2.clean_comment_nemoji
Y_test = df2.off
# splitting into train and tests
#X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

In [71]:
Tfidf_vect = TfidfVectorizer()


In [72]:
Tfidf_vect.fit(df1["clean_comment_nemoji"])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [73]:
print(Train_X_Tfidf.shape)
print(Test_X_Tfidf.shape)

(6419, 23746)
(1605, 23746)


In [74]:
tf_idf_vec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=20000)

<h1> logistic regression </h1>

In [16]:
# make pipeline
pipe = make_pipeline(tf_idf_vec, LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
model = GridSearchCV(pipe, param_grid, cv=10, verbose=1)
model.fit(X_train,Y_train)

# make prediction and print accuracy
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   16.3s finished


Accuracy score is 0.78
              precision    recall  f1-score   support

           0     0.7460    0.7782    0.7618       717
           1     0.8145    0.7860    0.8000       888

    accuracy                         0.7826      1605
   macro avg     0.7802    0.7821    0.7809      1605
weighted avg     0.7839    0.7826    0.7829      1605



In [17]:
model.best_params_

{'logisticregression__C': 10}

In [18]:
model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

In [19]:
model.predict([clean('زي الهوا يا حبيبي زي الهوا')])

array([0], dtype=int64)

<h1> SVM </h1>


In [20]:
pipe = make_pipeline(tf_idf_vec, SVC())
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

svc_model = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
svc_model.fit(X_train, Y_train)

prediction = svc_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 13.0min finished


Accuracy score is 0.79
              precision    recall  f1-score   support

           0     0.7527    0.7727    0.7626       717
           1     0.8124    0.7950    0.8036       888

    accuracy                         0.7850      1605
   macro avg     0.7826    0.7839    0.7831      1605
weighted avg     0.7858    0.7850    0.7853      1605



In [21]:
svc_model.best_params_

{'svc__C': 100, 'svc__gamma': 1, 'svc__kernel': 'rbf'}

In [22]:
svc_model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

<h1>  Random Forest Model </h1>


In [75]:
pipe = make_pipeline(tf_idf_vec, RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 17.6min finished


Accuracy score is 0.78


In [76]:
print(classification_report(Y_test, prediction, digits=4))

              precision    recall  f1-score   support

           0     0.7450    0.7866    0.7653       717
           1     0.8196    0.7827    0.8007       888

    accuracy                         0.7844      1605
   macro avg     0.7823    0.7846    0.7830      1605
weighted avg     0.7863    0.7844    0.7849      1605



In [24]:
print(classification_report(Y_test, prediction, digits=4))

              precision    recall  f1-score   support

           0     0.7477    0.7852    0.7660       717
           1     0.8192    0.7860    0.8023       888

    accuracy                         0.7857      1605
   macro avg     0.7835    0.7856    0.7841      1605
weighted avg     0.7873    0.7857    0.7861      1605



In [25]:
rf_model.best_params_

{'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__n_estimators': 1000}

In [26]:
rf_model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

## Naive Bayes Model

In [27]:
pipe = make_pipeline(tf_idf_vec, MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Accuracy score is 0.79
              precision    recall  f1-score   support

           0     0.8205    0.6695    0.7373       717
           1     0.7676    0.8818    0.8208       888

    accuracy                         0.7869      1605
   macro avg     0.7941    0.7756    0.7790      1605
weighted avg     0.7913    0.7869    0.7835      1605



In [29]:
pipe.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

# With emojis

In [30]:
from emoji import UNICODE_EMOJI

In [31]:
def is_emoji(s):
    return s in UNICODE_EMOJI['en']
# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

In [32]:
df1["clean_comment_emoji"]=df1["clean_comment"].apply(lambda row: add_space(row))
df2["clean_comment_emoji"]=df2["clean_comment"].apply(lambda row: add_space(row))

In [33]:
df1

,Unnamed: 0,comment,off,clean_comment,clean_comment_nemoji,clean_comment_emoji
0,2908,فنانين الكبت والفساد .عقلية جنسية لا غير. العف...,1,فنانين الكبت والفساد عقليه جنسيه العفن الفن شع...,فنانين الكبت والفساد عقليه جنسيه العفن الفن شع...,فنانين الكبت والفساد عقليه جنسيه العفن الفن شع...
1,1055,الدعارة هربت منها في المحمدية و سكنت في بوزنيق...,1,الدعاره هربت المحمديه سكنت بوزنيقه بحال بحال م...,الدعاره هربت المحمديه سكنت بوزنيقه بحال بحال م...,الدعاره هربت المحمديه سكنت بوزنيقه بحال بحال م...
2,181,كون غير خريتي و مدرتيش هادشي,1,كون خريتي مدرتيش هادشي,كون خريتي مدرتيش هادشي,كون خريتي مدرتيش هادشي
3,4313,لا حول ولا قوة الا بالله العلي العظيم لا حول و...,0,بالله العلي العظيم بالله العلي العظيم استغفرال...,بالله العلي العظيم بالله العلي العظيم استغفرال...,بالله العلي العظيم بالله العلي العظيم استغفرال...
4,228,الله يرحم الوالدين عندي مشكل كبيييير و محتاجة ...,0,الله يرحم الوالدين عندي مشكل كبيير محتاجه الدع...,الله يرحم الوالدين عندي مشكل كبيير محتاجه الدع...,الله يرحم الوالدين عندي مشكل كبيير محتاجه الدع...
...,...,...,...,...,...,...
6414,79,هدى جمعاك شعر كحل,0,هدي جمعاك شعر كحل,هدي جمعاك شعر كحل,هدي جمعاك شعر كحل
6415,3927,تفو.. على أغنية زملاوية بحال مولاها,1,تفو اغنيه زملاويه بحال مولاها,تفو اغنيه زملاويه بحال مولاها,تفو اغنيه زملاويه بحال مولاها
6416,5955,كون خرجات ها.ي كالت انا كنشطح كون بان ليك مساع...,1,كون خرجات هاي كالت انا كنشطح كون ليك مساعادات ...,كون خرجات هاي كالت انا كنشطح كون ليك مساعادات ...,كون خرجات هاي كالت انا كنشطح كون ليك مساعادات ...
6417,6936,الفديو زوين.,0,الفديو زوين,الفديو زوين,الفديو زوين


In [34]:
'''feature = df.clean_comment_emoji
target = df.off'''
X_train = df1.clean_comment_emoji
Y_train = df1.off
X_test = df2.clean_comment_emoji
Y_test = df2.off
# splitting into train and tests
#X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

In [35]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df1["clean_comment_emoji"])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [36]:
tf_idf_vec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=20000)

<h1> logistic regression </h1>

In [37]:
# make pipeline
pipe = make_pipeline(tf_idf_vec, LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
model = GridSearchCV(pipe, param_grid, cv=10, verbose=1)
model.fit(X_train,Y_train)

# make prediction and print accuracy
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   24.5s finished


Accuracy score is 0.78
              precision    recall  f1-score   support

           0     0.7470    0.7782    0.7623       717
           1     0.8147    0.7872    0.8007       888

    accuracy                         0.7832      1605
   macro avg     0.7808    0.7827    0.7815      1605
weighted avg     0.7844    0.7832    0.7835      1605



In [38]:
model.best_params_

{'logisticregression__C': 10}

In [39]:
model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

In [40]:
model.predict([clean('زي الهوا يا حبيبي زي الهوا')])

array([0], dtype=int64)

<h1> SVM </h1>


In [41]:
pipe = make_pipeline(tf_idf_vec, SVC())
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

svc_model = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
svc_model.fit(X_train, Y_train)

prediction = svc_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 21.2min finished


Accuracy score is 0.79
              precision    recall  f1-score   support

           0     0.7531    0.7741    0.7634       717
           1     0.8134    0.7950    0.8041       888

    accuracy                         0.7857      1605
   macro avg     0.7832    0.7846    0.7838      1605
weighted avg     0.7864    0.7857    0.7859      1605



In [42]:
print(classification_report(Y_test, prediction, digits=4))

              precision    recall  f1-score   support

           0     0.7531    0.7741    0.7634       717
           1     0.8134    0.7950    0.8041       888

    accuracy                         0.7857      1605
   macro avg     0.7832    0.7846    0.7838      1605
weighted avg     0.7864    0.7857    0.7859      1605



In [43]:
svc_model.best_params_

{'svc__C': 100, 'svc__gamma': 1, 'svc__kernel': 'rbf'}

In [44]:
svc_model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

<h1>  Random Forest Model </h1>


In [45]:
pipe = make_pipeline(tf_idf_vec, RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe, param_grid, cv=5, verbose=1)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.7min finished


Accuracy score is 0.78


In [46]:
print(classification_report(Y_test, prediction, digits=4))

              precision    recall  f1-score   support

           0     0.7440    0.7741    0.7587       717
           1     0.8114    0.7849    0.7979       888

    accuracy                         0.7801      1605
   macro avg     0.7777    0.7795    0.7783      1605
weighted avg     0.7813    0.7801    0.7804      1605



In [47]:
rf_model.best_params_

{'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__n_estimators': 1000}

In [48]:
rf_model.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)

## Naive Bayes Model

In [49]:
pipe = make_pipeline(tf_idf_vec, MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction, digits=4))

Accuracy score is 0.79
              precision    recall  f1-score   support

           0     0.8205    0.6695    0.7373       717
           1     0.7676    0.8818    0.8208       888

    accuracy                         0.7869      1605
   macro avg     0.7941    0.7756    0.7790      1605
weighted avg     0.7913    0.7869    0.7835      1605



In [51]:
pipe.predict([clean('الزمر فالطوندونس هو باطمة تلاقيتو تواتيتو')])

array([1], dtype=int64)